# Importing library and data

In [1]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from collections import Counter
import warnings
import folium 
import matplotlib.pyplot as plt

#Set options
warnings.filterwarnings('ignore')
pd.set_option('max_columns',500)
%matplotlib inline

In [2]:
#Load data
df = pd.read_csv('crash.csv')
print('complete data size: ', len(df))

#Sample 5% of data
df = df.sample(frac=.05)

#Print header
print('Finished loading')
display(df.head(1))

complete data size:  1656944
Finished loading


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
1562360,01/10/2013,12:00,NaN,NaN,40.61847,-73.897451,POINT (-73.8974505 40.6184698),NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,2950090,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN


# Initial understanding of the data & filtering out noise

The sampled data runs from 2012-07-01 to 2020-02-24

In [179]:
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
print('The earliest incident: ', min(df['CRASH DATE']))
print('The latest incident: ', max(df['CRASH DATE']))

The earliest incident:  2012-07-01 00:00:00
The latest incident:  2020-02-24 00:00:00


## Creating visualization of crashes

There are records where NYC input latitude & longitude in Asia/Africa. Those are obviously errors. Let's use a filter to cut them off.

In [3]:
print('We originally have sampled {} data points'.format(len(df)))

#Creating a list of locations (in the format of list of list)
df = df.dropna(subset = ['LATITUDE','LONGITUDE'])
df = df[
    (41.2 > df['LATITUDE'])&
    (df['LATITUDE']> 40.2) &
    (-73 > df['LONGITUDE']) &
    (df['LONGITUDE'] > -75)
]
locations = df[['LATITUDE','LONGITUDE']]
location_list = locations.values.tolist() 

#Print info regarding the crashes
print('After dropping null values & applying filter, we have {} data points'.format(len(location_list)))

We originally have sampled 82847 data points
After dropping null values & applying filter, we have 72774 data points


In [4]:
#4 corners of the geo-filter
points = [[41.2, -75],[40.2, -75], [41.2,-73],[40.2,-73],]

#Only using 1000 crashes for visualization
map = folium.Map(location=[40.7,-74], tiles='Stamen Toner', zoom_start=9)
for point in range(0,1000):
    folium.CircleMarker(location_list[point], radius=3, color='red', fill=True, 
                        fill_color='red', opacity = 0.4).add_to(map)

#Plotting out the rectangle (geographic-filter)
folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(map)
map

#  Accidents rate analysis by geographical lcoation

Key take away: It's easiest to get into an accident in Manhattan than any other boroughs.

1. Brooklyn has the most accidents over the last 8 year (17537 cases), while Staten Island has the least number of accidents (2369 cases). If we were to randomly drive acrosss NYC, we would be ~6.4 times more likely to get into an accident in Brooklyn than in Staten Island. **17537/2368 = 7.4**

2. After normalizing the data by area, Manhattan has 580 accidents per sq-mile over the last 8 years, while Staten island has 40 accidents per sq-mile over the last 8 years. Given the fact that person A & person B are driving the same amount of distance in Manhattan & Staten island, respectively, person A is 13.5 times more likely to get into an accident than person B. **580/40 = 14.5 x 8**

3. If one is to get into an accident, there is a ~30% he will get injuried if he is driving in Brooklyn or Bronx. There is only a 17% chance he will get injuried if he is driving in Manhattan.

4. There are not enough data to make any claims about deadly accidents. For example:

Bronx: 8 deaths, A sample size of 8 is not adequate to make any meaningful claims.

We may wish to further investigate by resampling a few more times, or increase the sample size drastically.

In [12]:
def investigate(x):
    grp= df.groupby(x)['NUMBER OF PERSONS INJURED'].agg(['mean','sum','count'])
    cases = grp['count']
    unidentified = len(df[df[x].isnull()])
    
    #Display
    print('total accidents in identified ' + x + 'S :', sum(cases))
    print('total accidents in unidentified ' + x + 'S :', unidentified)
    print('total accidents: ', len(df))
    
    return grp

In [14]:
grp = investigate('BOROUGH')
grp.columns = ['injury rate','total injuries','total accidents']
grp['area sq miles'] = [42.47, 69.5, 22.82, 108.1, 58.69]
grp['total accidents per sq miles 2012-2020'] = grp['total accidents']/grp['area sq miles']
grp['total injuries per sq miles 2012-2020'] = grp['total injuries']/grp['area sq miles']
grp

total accidents in identified BOROUGHS : 56153
total accidents in unidentified BOROUGHS : 16621
total accidents:  72774


,injury rate,total injuries,total accidents,area sq miles,total accidents per sq miles 2012-2020,total injuries per sq miles 2012-2020
BOROUGH,,,,,,
BRONX,0.277984,2173.0,7817,42.47,184.059336,51.165529
BROOKLYN,0.292182,5124.0,17537,69.50,252.330935,73.726619
MANHATTAN,0.167749,2222.0,13246,22.82,580.455741,97.370727
QUEENS,0.268836,4082.0,15184,108.10,140.462535,37.761332
STATEN ISLAND,0.264247,626.0,2369,58.69,40.364628,10.666212


## Investigating the most dangerous zip codes

In [7]:
#Create df
zip_grp = investigate('ZIP CODE')

#Filter out df to only have zip codes that have normal/above normal # of accidents
expected = zip_grp['count'].sum()/len(zip_grp)
zip_grp = zip_grp[zip_grp['count']>=expected]
zip_grp = zip_grp.reset_index()

#Select a random latitude & longitude value for each zip code, use new_df for plotting
zip_df = df.drop_duplicates(subset='ZIP CODE')
zip_df = zip_df[['ZIP CODE', 'LATITUDE', 'LONGITUDE']]
zip_plot_df = zip_df.merge(zip_grp, how = 'inner', on = 'ZIP CODE')
zip_plot_df = zip_plot_df.sort_values(by = 'count', ascending = False).reset_index()
zip_plot_df
print()

total accidents in identified ZIP CODES : 56148
total accidents in unidentified ZIP CODES : 16626
total accidents:  72774



## Visualizing the accidents by zip-codes

The top 10 most dangerous zip-codes are colored in blue in the map. Interestingly, here are some common traits these zip-codes share.

1. They are in Manhattan or near the bridge from one Borough to Manhattan

2. They are near the NY-27 high way, which connects NYC to the more 'sub-urban' side of NY.

3. They are near airports.

From these observations, we may be able to make some educated guesses on the cause of high accident rates. People often get into accidents due to rushing (traveling to Manhattan, or traveling to JFK). Sub-urban residents may also cause accidents due to inexperience of driving in the city.

In [8]:
#Create attributes for plotting
zip_stat = zip_plot_df['count'].apply(lambda x: x*0.025)
zip_caption = zip_plot_df['ZIP CODE'].apply(lambda x: str(int(x)))
zip_location = zip_plot_df[['LATITUDE','LONGITUDE']]
zip_location = zip_location.values.tolist()

#Plotting the map
map = folium.Map(location=[40.7,-74], tiles='Stamen Toner', zoom_start=11)
for point in range(0,len(zip_location)):
    pt_stat = float(zip_stat[point])
    #Plot top 10 zipcodes in blue, recall we sort them in descending order already
    if point < 10:
        folium.CircleMarker(zip_location[point], radius=pt_stat, color='black', fill=True, 
                            fill_color='blue', 
                            fill_opacity = 1).add_child(folium.Popup(zip_caption[point])).add_to(map)
    else:
        folium.CircleMarker(zip_location[point], radius=pt_stat, color='black', fill=True, 
                            fill_color='red', 
                            fill_opacity = 0.5).add_child(folium.Popup(zip_caption[point])).add_to(map)
map

# DBSCAN
Disadvantages of K-means in this project

We shall use a density-based clustering method instead of K-means for this project. I have performed K-means on this project, and found several disadvantages.

1. The elbow method suggested to use 5 clusters, and we were able to find 5 clusters based on this method. This method gave a output of 5 geo-clusters: Staten Island, Queens, downtown Brooklyn, Bronx, (Manhattan & uptown Brooklyn). This method gives us clusters based on centroid calculations instead of man-made geo-labels. However, this method doesn't give any interesting insghts.

2. If we were to use more clusters, K-means may not skip outliers.

3. It may be difficult to determine to optimal number of clusters to use if our main intention is to look at much smaller clusters.

Below is a snippet of the K-means clustering results.
![title](K-means.png)

**The need for a clustering technique**

Although it's intuitive to compare accident rates in zip-code, or boroughs. They may not be the most technically correct way to find areas with highest accident rates. There may be cases where 2 accidents are very close to each other, but they share different zipcodes. 

I used very strict parameters for the DBSCAN, and iterated the process until I am pleased with the results. I found it is difficult to decide the optimal parameters to use based on our intention. However, we always find downtown manhattan to the most dangerous place to drive. **This finding actually aligns with our zipcode, borough, K-means study part.**


If we look at the zip code part, we can find many large zip-code bubbles existing in Manhattan.

K-means square also suggest that (lower Manhattan & upper Brooklyn) have the most cases.

In [9]:
#make df for DBSCAN
db_df = locations[0:2000]

#make location list
db_array = db_df.values.astype('float32')
len(db_array)

2000

In [10]:
#Construct model
'''
eps = maximum distance btw 2 samples for one to be considered in the neighborhood of another
min_samples = The number of samples (or total weight) in a neighborhood for a point to be considered as a core point. 

1 degree longitude ~ 54.6 mile
1 degree latitude ~ 69 miles

To achieve better technical accuracy, we may wish to do a weighted distance difference when calculating the 
euclidean distance. 

eps = 0.01 is a rough estimate of 0.9 miles
'''

model = DBSCAN(eps = 0.01, min_samples = 35, metric = 'euclidean').fit(db_array)
db_df['labels'] = model.labels_
display(db_df.head())

# Get info about the clusters
clusters = Counter(model.labels_)
print(clusters)
print('Number of clusters = {}'.format(len(clusters)-1))

,LATITUDE,LONGITUDE,labels
1562360,40.618470,-73.897451,-1
189316,40.804325,-73.966606,-1
1245604,40.686117,-73.869910,-1
910801,40.599550,-73.761288,-1
316178,40.829002,-73.915570,-1


Counter({-1: 1766, 0: 234})
Number of clusters = 1


In [11]:
#return color function
def return_color(x):
    color_list = ['red', 'blue', 'green', 'purple',
    'orange', 'darkred', 'lightred', 'beige', 'darkblue', 
    'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink',
    'lightblue', 'lightgreen', 'gray', 'lightgray']*10
    
    if x == -1:
        return 'black'
    else:
        return color_list[x]

#plot map
map = folium.Map(location=[40.7,-74], tiles='Stamen Toner', zoom_start=11)
for point in range(0,len(db_array)):
    c_index = model.labels_[point]
    folium.CircleMarker(db_array[point], radius=5, color=return_color(c_index), fill=True, 
                        fill_color=return_color(c_index), opacity = 1, fill_opacity = 1).add_to(map)
map